[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/enterprise/healthcare/BioNLP_NER.ipynb)

## BioNLP Named Entity Recognition

In [1]:
import json

with open('keys.json') as f:
    license_keys = json.load(f)

license_keys.keys()

dict_keys(['version', 'jsl_version', 'secret', 'SPARK_NLP_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'JSL_OCR_LICENSE', 'JSL_OCR_SECRET'])

In [2]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

secret = license_keys['secret']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['JSL_OCR_LICENSE'] = license_keys['JSL_OCR_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
version = license_keys['version']
jsl_version = license_keys['jsl_version']

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl



def start(secret):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:"+version) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-"+jsl_version+".jar")
      
    return builder.getOrCreate()


spark = start(secret) # if you want to start the session with custom params as in start function above
# sparknlp_jsl.start(secret)

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/scwgF2mD1U
     |████████████████████████████████| 215.7MB 52kB/s 
     |████████████████████████████████| 133kB 36.9MB/s 
     |████████████████████████████████| 204kB 31.1MB/s 
  Created wheel for spark-nlp-jsl: filename=spark_nlp_jsl-2.5.4rc2-cp36-none-any.whl size=22983 sha256=8d3a831e5323b4c891a6832469c3ced5e3774eecba4fe997fbc1fdaa228c66b0
  Stored in directory: /root/.cache/pip/wheels/17/aa/21/fd766748d93cdb7e75f311749ee5e90cc531837704e182302e
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=e8f9ac73a111430b065a343571cf6677c30a73e398a4f03bd84d5ac6d25051ab
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built spark-nlp-jsl pyspark
2.5.4


In [3]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

clinical_pos = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token"]) \
  .setOutputCol("pos")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu") \
  .setInputCols(["sentence","token", "pos"]) \
  .setOutputCol("dependency")

bio_ner = NerDLModel.pretrained('ner_bionlp', 'en', 'clinical/models')

converter = NerConverter()\
  .setInputCols(["sentence", "token", "ner"])\
  .setOutputCol("ner_span")

clinical_ner_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        embeddings,
        clinical_pos,
        dependency_parser,
        bio_ner,
        converter])


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.7 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.6 MB
[OK!]
ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[OK!]


In [4]:
import pyspark.sql.functions as F

text="""
Stage 4 adenocarcinoma of lung with b/l lung nodules , probable L-2 metastasis and a negative brain MRI . Molecular testing of tumor demonstrates EGFR mutation . On Tarceva . Disease Stable . Plan Chemotherapy toxicity reviewed again . Patient agrees to proceed . Counseling time : 40 mins . 1 ) Continue Tarceva for lung cancer . CT CAP ordered before next visit 2 ) Continue anti-anxiety medication 3 ) Xgeva today 4 ) Return in 1 month with labs , after scan
"""

prediction_data = spark.createDataFrame([[text]]).toDF("text")

prediction_data.show(truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------

In [5]:

prediction_model=clinical_ner_pipeline.fit(prediction_data)

preds = prediction_model.transform(prediction_data)

preds.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities")) \
.select(F.expr("entities['0']").alias("chunk"),
        F.expr("entities['1'].entity").alias("entity"))\
        .show(truncate=False)

+--------------+----------------------+
|chunk         |entity                |
+--------------+----------------------+
|adenocarcinoma|Cancer                |
|lung          |Organ                 |
|lung nodules  |Pathological_formation|
|L-2           |Cell                  |
|brain         |Organ                 |
|tumor         |Cancer                |
|EGFR          |Gene_or_gene_product  |
|Tarceva       |Simple_chemical       |
|Patient       |Organism              |
|lung cancer   |Cancer                |
+--------------+----------------------+

